<a href="https://colab.research.google.com/github/Joshua-Pow/pokescanner/blob/main/PokeScanner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 1. Importing libraries / Data Loading and Splitting



In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import matplotlib.pyplot as plt

from torchvision import transforms

from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

transform = transforms.Compose([
    # you can add other transformations in this list
    transforms.ToTensor()
])

### 2. Model Building and Sanity Checking

Build a convolutional neural network model that takes the (224x224 RGB) image as input, and predicts the pokemon

In [ ]:
class SignLanguageClassifier(nn.Module):
      def __init__(self, kernel_size):
          super(SignLanguageClassifier, self).__init__()
          self.conv1 = nn.Conv2d(3, 2, kernel_size) #in_channels, out_chanels, kernel_size
          self.pool = nn.MaxPool2d(2, 2) #kernel_size, stride 
          self.conv2 = nn.Conv2d(2, 2, kernel_size) #in_channels, out_chanels, kernel_size
          self.fc1 = nn.Linear(5618, 32) #needs input-dependent value
          self.fc2 = nn.Linear(32, 9)

      def forward(self, x):
          x = self.pool(F.relu(self.conv1(x)))
          x = self.pool(F.relu(self.conv2(x)))
          x = x.view(-1, 5618)
          x = F.relu(self.fc1(x))
          x = self.fc2(x)
          return x

In [ ]:
#Hello World
#Test Line